In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# The link og the dataset : https://www.kaggle.com/datasets/krantiswalke/bankfullcsv
df = pd.read_csv("../input/bankfullcsv/bank-full.csv")
df.head(3)

In [3]:
df.describe()

In [4]:
df.info()

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
columns_to_drop = ['job', 'marital', 'education', 'month', 'contact', 'poutcome', 'housing', 'loan', 'Target', 'default']
for colum_to_drop in columns_to_drop:
    encoder.fit(np.array(df[colum_to_drop]))
    df[colum_to_drop]=encoder.transform(df[colum_to_drop])

In [6]:
df.head()

In [7]:
colums_have_unknow=['job', 'marital', 'education', 'default', 'housing', 'loan']
for colum_has_unknow in colums_have_unknow:
    df.drop(df[df[colum_has_unknow] == 'unknow'].index, inplace = True)
df.info()

In [8]:
y = df["Target"].copy()
x = df.drop("Target", axis=1)

In [9]:
plt.figure(figsize=(17,17))
sns.heatmap(data = df.corr(), annot=False)
plt.show()

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 42)

In [11]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

In [12]:
log_clf = LogisticRegression(solver='lbfgs', max_iter=30000)
dt_clf = DecisionTreeClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', dt_clf), ('svc', svm_clf)])
voting_clf.fit(x_train, y_train)

In [13]:
result = {}
from sklearn.metrics import accuracy_score
for clf in (log_clf, dt_clf, svm_clf, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    result[clf.__class__.__name__]=accuracy_score(y_test, y_pred)

In [14]:
result

In [15]:
# soft voting
log_clf = LogisticRegression(solver='lbfgs', max_iter=30000)
dt_clf = DecisionTreeClassifier()
svm_clf = SVC(probability=True)
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', dt_clf), ('svc', svm_clf)], voting='soft')
voting_clf.fit(x_train, y_train)

In [16]:
result = {}
from sklearn.metrics import accuracy_score
for clf in (log_clf, dt_clf, svm_clf, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    result[clf.__class__.__name__]=accuracy_score(y_test, y_pred)

In [17]:
result

In [18]:
#pasting
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=2000, max_samples=400, bootstrap=False, n_jobs=-1)
bag_clf.fit(x_train, y_train)
y_pred = bag_clf.predict(x_test)
accuracy_score(y_test, y_pred)

In [19]:
#bagging
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=2000, max_samples=400, bootstrap=True, n_jobs=-1)
bag_clf.fit(x_train, y_train)
y_pred = bag_clf.predict(x_test)
accuracy_score(y_test, y_pred)

In [20]:
#oob
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=2000, max_samples=400, bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(x_train, y_train)
y_pred = bag_clf.predict(x_test)
bag_clf.oob_score_

In [21]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(x_train, y_train)
y_pred_rf = rnd_clf.predict(x_test)
accuracy_score(y_test, y_pred)